In [1]:
from toolbox import *
from music21 import *

In [2]:
file = converter.parse('stalker.mid')


In [3]:
# for track in len(file:
for i in range(len(file.parts)):
    print(len(file.parts[i]))

77
73
51
77
77
43
58
77
73
76
77
77
77
77
77
57
77


NoneType

In [5]:
for part in file.measure(2):
    print(get_list_entropy(get_pitches(part)))

0.0
0.6365141682948128
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0


In [14]:
def get_entropy_from_measure(file,track,measure):
    E_p = get_pitches(file.measure(measure).parts[track])
    E_r = get_rhythm(file.measure(measure).parts[track])
    E_s = get_rests(file.measure(measure).parts[track])
    total_entropy = get_list_entropy(E_p) + get_list_entropy(E_r)+ get_list_entropy(E_s)
    return total_entropy

In [15]:

get_entropy_from_measure(converter.parse('stalker.mid'),1,2)

1.2730283365896256

In [9]:
file.measure(1).parts[2]

<music21.stream.Part 0x7fb3a99f4850>

In [50]:
from pyqubo import Binary, Constraint


def max_num_measures(file):
    return max([len(p) for p in file.parts])
    

def add_measurecountrains_from(file):
    H = 0
    for j in range(50,max_num_measures(file)):
        c = 0 
        for i in range(len(file.parts)):
#             print(i,j, file.measure(j).parts[i])
            if file.parts[i].measure(j) != None:
                c += Binary(f"x_{i}_{j}")
        H += Constraint((2-c)**2, f"measure_{j}")
    return H
        
# def qubo_form

In [51]:
add_countrains_from(file)

(Constraint(label=measure_76,((Binary(x_16_76)+Binary(x_14_76)+Binary(x_13_76)+Binary(x_12_76)+Binary(x_11_76)+Binary(x_10_76)+Binary(x_9_76)+Binary(x_7_76)+Binary(x_4_76)+Binary(x_0_76)+Binary(x_3_76))*Num(-1.000000)+Num(2.000000))*((Binary(x_16_76)+Binary(x_14_76)+Binary(x_13_76)+Binary(x_12_76)+Binary(x_11_76)+Binary(x_10_76)+Binary(x_9_76)+Binary(x_7_76)+Binary(x_4_76)+Binary(x_0_76)+Binary(x_3_76))*Num(-1.000000)+Num(2.000000)))+Constraint(label=measure_75,((Binary(x_16_75)+Binary(x_14_75)+Binary(x_13_75)+Binary(x_12_75)+Binary(x_11_75)+Binary(x_10_75)+Binary(x_9_75)+Binary(x_7_75)+Binary(x_4_75)+Binary(x_0_75)+Binary(x_3_75))*Num(-1.000000)+Num(2.000000))*((Binary(x_16_75)+Binary(x_14_75)+Binary(x_13_75)+Binary(x_12_75)+Binary(x_11_75)+Binary(x_10_75)+Binary(x_9_75)+Binary(x_7_75)+Binary(x_4_75)+Binary(x_0_75)+Binary(x_3_75))*Num(-1.000000)+Num(2.000000)))+Constraint(label=measure_74,((Binary(x_16_74)+Binary(x_14_74)+Binary(x_13_74)+Binary(x_12_74)+Binary(x_11_74)+Binary(x_10_74)

In [39]:
file.measure(53).parts[2]

<music21.stream.Part 0x7fb3a7003c40>

In [38]:
file.parts[2].measure(52)